In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import warnings
a=pd.read_csv(r"C:\\Users\Graziano\Desktop\mio\catalogue\DATA3_magn.csv")
#put the index in ascending way
a.sort_values(by='Time',ascending=False,inplace=True) #sort values according to time
num=pd.Series([i for i in reversed(range(0,int(a.size/len(a.columns))))])  #range(0,2,-1) gives [2,1], riversed gives [1,0]
a=pd.concat([a,num],axis=1)
a.set_index(0,inplace=True) #remember inplace
a.iloc[::-1]
a

,#EventID,Time,Latitude,Longitude,Depth/Km,Author,Catalog,Contributor,ContributorID,MagType,Magnitude,MagAuthor,EventLocationName
0,,,,,,,,,,,,,
0,24328471,2020-04-28T13:00:30.050000,38.4607,13.64330,17.7,SURVEY-INGV,NaN,NaN,NaN,ML,3.5,--,Tirreno Meridionale (MARE)
1,24326871,2020-04-28T07:14:45.750000,38.5943,14.17220,27.5,SURVEY-INGV,NaN,NaN,NaN,ML,3.0,--,Tirreno Meridionale (MARE)
2,24326451,2020-04-28T05:57:28.820000,38.4052,13.60600,25.4,SURVEY-INGV,NaN,NaN,NaN,ML,3.1,--,Costa Siciliana centro-settentrionale (Palermo)
3,24317311,2020-04-26T02:59:02.450000,40.8285,14.15167,2.5,SURVEY-INGV-OV,NaN,NaN,NaN,Md,3.1,--,5 km E Pozzuoli (NA)
4,24313381,2020-04-25T07:21:54.480000,37.7523,14.61750,23.5,BULLETIN-INGV,NaN,NaN,NaN,ML,3.4,--,4 km SE Troina (EN)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11189,529,1985-01-07T00:34:49.290000,43.2250,12.73000,10.0,BULLETIN-VAX,NaN,NaN,NaN,Md,3.0,--,5 km W Gualdo Tadino (PG)
11190,489,1985-01-06T17:22:58.540000,39.2910,16.76500,5.0,BULLETIN-VAX,NaN,NaN,NaN,Md,3.0,--,2 km N Castelsilano (KR)
11191,239,1985-01-05T03:47:03.640000,45.4090,6.62700,10.0,BULLETIN-VAX,NaN,NaN,NaN,M,3.2,--,Confine Italia-Francia (FRANCIA)


In [ ]:
'''def funct_lat_long(i,j):
    #timei=a_time.iloc[i+j]
    dtime=((timej-a_time.iloc[i+j])).total_seconds()
    lati = radians(a.iloc[i+j]['Latitude'])
    loni = radians(a.iloc[i+j]['Longitude'])
    a2 = sin((lati-latj) / 2)**2 + cos(latj) * cos(lati) * sin((loni-lonj) / 2)**2
    distance = R * (sqrt((2 * atan2(sqrt(a2), sqrt(1 - a2)))**2+(a.iloc[i+j]['Depth/Km']-a.iloc[j]['Depth/Km'])**2))
    space_time=round(dtime*distance**d*pow(10,-b*float(a.iloc[i+j]['Magnitude'])),12)
    values.append(space_time)
    return values, space_time, distance'''
#a.size: size of data frame, a.columns: size of the columns
#access the first (only) row as a Series using iloc, and then the value using the column name

### ALGORITHM
#### (There is the possibility that you couldn't be able to run the algorithm for the first time- In this case, try again).
#### It takes a long time to have the results: at the moment don't put more than 1000 events in a catalogue or it could take several hours to spit up the solution.
#### the result is a list with n lists nested (n is the number of events taken). In this "nested list" there are 3 elements: the first one is the $N_{ij}$ parameter, the second is the position of the mainshock in the catalogue, and the third one is the position of the aftershock.

In [44]:
from datetime import datetime
import time
from math import sin, cos, sqrt, atan2, radians, pow
d=1.6  
b=1.21
a_time=pd.Series(a['Time'])
a['Time']=a_time.astype(np.datetime64)
space_time_total=[]
start_time=time.time()
R = 6373.0    # approximate earth radius in km
for j in range(int(a.size/len(a.columns))-3):  
    timej=a_time.iloc[j]  
    latj = radians(a.iloc[j]['Latitude'])
    lonj = radians(a.iloc[j]['Longitude'])
    i=1
    values=[]
    while i+j <= int(a.size/len(a.columns))-2 : 
        #timei=a_time.iloc[i+j]
        dtime=((timej-a_time.iloc[i+j])).total_seconds()
        lati = radians(a.iloc[i+j]['Latitude'])
        loni = radians(a.iloc[i+j]['Longitude'])
        a2 = sin((lati-latj) / 2)**2 + cos(latj) * cos(lati) * sin((loni-lonj) / 2)**2
        distance = R*(sqrt((2 * atan2(sqrt(a2), sqrt(1 - a2)))**2+
            (a.iloc[i+j]['Depth/Km']-a.iloc[j]['Depth/Km'])**2))
        space_time=round(dtime*distance**d*pow(10,-b*float(a.iloc[i+j]['Magnitude'])),12)
        values.append(space_time)
        i+=1
    min_value=min(values)    
    i_value=values.index(min_value)+1+j
    space_time_total.append([min_value,i_value,j])
end_time=time.time()
total_time=end_time-start_time
print(space_time_total,total_time)


[[28854386.69198039, 1907, 0], [3537832.369221905, 2, 1], [8460595.1654502, 9, 2], [35316376.37629168, 6208, 3], [2325440.926930323, 36, 4], [21572849.029225595, 23, 5], [358940.0695735319, 3045, 6], [18161449.493397087, 1973, 7], [885117.7500843582, 13, 8], [6608620.671072653, 394, 9], [2647141.7704029223, 389, 10], [387519.15322987194, 2065, 11], [106162.65938668484, 14, 12], [5076695.296603572, 14, 13], [3281327.999418661, 3238, 14], [20219356.003785945, 31, 15], [1018518.9004728578, 83, 16], [493830.9356931755, 21, 17], [195703.51048578316, 34, 18], [579240.3406658623, 300, 19], [710194.0286981624, 21, 20], [3154841.8686913806, 22, 21], [1929354.0845542634, 23, 22], [82145518.30683464, 31, 23], [14016533.924754728, 31, 24], [422722.7947802585, 90, 25], [10535020.3062035, 245, 26], [3942.100864274345, 31, 27], [152861.50368188103, 31, 28], [254618.05654967867, 31, 29], [301.989933340664, 31, 30], [52856.645296751456, 1984, 31], [15384.816093542404, 684, 32], [82897.808607199, 1486, 

In [46]:
print(values,total_time)

[314174597.23421323, 379745.1632048526] 99269.40620446205
